# Web Mining and Applied NLP (44-620)

## Requests, JSON, and NLP

### Student Name:[Jose Guzman](https://github.com/jg2012/json-sentiment/tree/master)

Perform the tasks described in the Markdown cells below.  When you have completed the assignment make sure your code cells have all been run (and have output beneath them) and ensure you have committed and pushed ALL of your changes to your assignment repository.

Make sure you have [installed spaCy and its pipeline](https://spacy.io/usage#quickstart) and [spaCyTextBlob](https://spacy.io/universe/project/spacy-textblob)

Every question that requires you to write code will have a code cell underneath it; you may either write your entire solution in that cell or write it in a python file (`.py`), then import and run the appropriate code to answer the question.

This assignment requires that you write additional files (either JSON or pickle files); make sure to submit those files in your repository as well.

In [3]:
# Create and activate a Python virtual environment. 
# Before starting the project, try all these imports FIRST
# Address any errors you get running this code cell 
# by installing the necessary packages into your active Python environment.
# Try to resolve issues using your materials and the web.
# If that doesn't work, ask for help in the discussion forums.
# You can't complete the exercises until you import these - start early! 
# We also import json and pickle (included in the Python Standard Library).

import json
import pickle

import requests
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

print('All prereqs installed.')
!pip list

All prereqs installed.
Package            Version
------------------ -----------
annotated-types    0.6.0
appnope            0.1.4
asttokens          2.4.1
beautifulsoup4     4.12.3
blis               0.7.11
catalogue          2.0.10
certifi            2024.2.2
charset-normalizer 3.3.2
click              8.1.7
cloudpathlib       0.16.0
comm               0.2.2
confection         0.1.4
cymem              2.0.8
debugpy            1.8.1
decorator          5.1.1
en-core-web-sm     3.7.1
executing          2.0.1
idna               3.6
ipykernel          6.29.3
ipython            8.22.2
jedi               0.19.1
Jinja2             3.1.3
joblib             1.3.2
jupyter_client     8.6.1
jupyter_core       5.7.2
langcodes          3.3.0
lyricsgenius       3.0.1
MarkupSafe         2.1.5
matplotlib-inline  0.1.6
murmurhash         1.0.10
nest-asyncio       1.6.0
nltk               3.8.1
numpy              1.26.4
packaging          24.0
parso              0.8.3
pexpect            4.9.0
pip       

1. The following code accesses the [lyrics.ovh](https://lyricsovh.docs.apiary.io/#reference/0/lyrics-of-a-song/search) public api, searches for the lyrics of a song, and stores it in a dictionary object.  Write the resulting json to a file (either a JSON file or a pickle file; you choose). You will read in the contents of this file for future questions so we do not need to frequently access the API.

In [36]:
import requests
import json

# Function to save JSON data to a file
def save_to_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)

# URL encode the song title and artist name
song_title = 'Ivy'
artist_name = 'Frank Ocean'
url = f'https://api.lyrics.ovh/v1/{artist_name}/{song_title.replace(" ", "%20")}'

# Fetch the lyrics data
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    result = json.loads(response.text)
    lyrics = result.get('lyrics', 'Lyrics not found')

    # Save the lyrics data to a JSON file
    filename = 'frank_ocean_ivy_lyrics.json'
    save_to_json(result, filename)
    print(f"Lyrics saved to '{filename}'")
else:
    print("Failed to retrieve lyrics. Error:", response.status_code)


Lyrics saved to 'frank_ocean_ivy_lyrics.json'


2. Read in the contents of your file.  Print the lyrics of the song (not the entire dictionary!) and use spaCyTextBlob to perform sentiment analysis on the lyrics.  Print the polarity score of the sentiment analysis.  Given that the range of the polarity score is `[-1.0,1.0]` which corresponds to how positive or negative the text in question is, do you think the lyrics have a more positive or negative connotaion?  Answer this question in a comment in your code cell.

In [37]:
import json
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize SpacyTextBlob with the loaded spaCy model
spacy_text_blob = SpacyTextBlob(nlp)

# Add SpacyTextBlob to the pipeline
nlp.add_pipe('spacytextblob')

# Function to perform sentiment analysis on the lyrics
def analyze_sentiment(lyrics):
    doc = nlp(lyrics)
    return doc._.polarity

# Read the contents of the JSON file
filename = 'frank_ocean_ivy_lyrics.json'
with open(filename, 'r') as json_file:
    data = json.load(json_file)
    lyrics = data['lyrics']

    # Print the lyrics
    print("Lyrics:")
    print(lyrics)
    print()

    # Perform sentiment analysis
    polarity_score = analyze_sentiment(lyrics)
    print("Polarity Score:", polarity_score)

    # Interpret polarity score
    # The polarity score ranges from -1.0 to 1.0
    # Negative polarity (close to -1.0) indicates negative sentiment,
    # Positive polarity (close to 1.0) indicates positive sentiment,
    # A polarity score around 0 indicates neutral sentiment.
    if polarity_score > 0:
        print("The lyrics have a more positive connotation.")
    elif polarity_score < 0:
        print("The lyrics have a more negative connotation.")
    else:
        print("The lyrics have a neutral connotation.")


Lyrics:
Paroles de la chanson Ivy par Frank Ocean
I thought that I was dreamin' when you said you love me
The start of nothin'
I had no chance to prepare, I couldn't see you comin'
The start of nothin'
Ooh, I could hate you now
It's quite alright to hate me now
When we both know that deep down
The feeling still deep down is good

If I could see through walls, I could see you're faking
If you could see my thoughts, you would see our faces
Safe in my rental like an armored truck back then

We didn't give a fuck back then
I ain't a kid no more
We'll never be those kids again
We'd drive to Syd's
Had the X6 back then, back then
No matter what I did, my waves wouldn't dip back then
Everything sucked back then, we were friends

I thought that I was dreamin' when you said you love me
The start of nothin'
I had no chance to prepare, I couldn't see you comin'
The start of nothin'
Ooh, I could hate you now
It's quite alright to hate me now
But we both know that deep down

The feeling still deep d

3. Write a function that takes an artist, song, and filename, accesses the lyrics.ovh api to get the song lyrics, and writes the results to the specified filename.  Test this function by getting the lyrics to any four songs of your choice and storing them in different files.

In [39]:
# Define the function to retrieve lyrics and save them to a JSON file
def get_lyrics_and_save(artist, song, filename):
    # Format the URL with the artist and song title
    url = f'https://api.lyrics.ovh/v1/{artist}/{song}'
    
    # Fetch the lyrics data
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        result = json.loads(response.text)
        lyrics = result.get('lyrics', 'Lyrics not found')
        
        # Write the lyrics to the specified JSON file
        with open(filename, 'w') as file:
            json.dump(result, file, indent=4)
        print(f"Lyrics saved to '{filename}'")
    else:
        print("Failed to retrieve lyrics. Error:", response.status_code)

# Retrieve lyrics for four Frank Ocean songs and save them into JSON files
get_lyrics_and_save("Frank Ocean", "Nikes", "frank_ocean_nikes_lyrics.json")
get_lyrics_and_save("Frank Ocean", "Nights", "frank_ocean_nights_lyrics.json")
get_lyrics_and_save("Frank Ocean", "Pyramids", "frank_ocean_pyramids_lyrics.json")
get_lyrics_and_save("Frank Ocean", "Pink + White", "frank_ocean_pink_white_lyrics.json")


Lyrics saved to 'frank_ocean_nikes_lyrics.json'
Lyrics saved to 'frank_ocean_nights_lyrics.json'
Lyrics saved to 'frank_ocean_pyramids_lyrics.json'
Lyrics saved to 'frank_ocean_pink_white_lyrics.json'


4. Write a function that takes the name of a file that contains song lyrics, loads the file, performs sentiment analysis, and returns the polarity score.  Use this function to print the polarity scores (with the name of the song) of the three files you created in question 3.  Does the reported polarity match your understanding of the song's lyrics? Why or why not do you think that might be?  Answer the questions in either a comment in the code cell or a markdown cell under the code cell.

In [40]:
from textblob import TextBlob

def analyze_sentiment_from_file(filename):
    # Read lyrics from the file
    with open(filename, 'r') as file:
        lyrics = file.read()
    
    # Perform sentiment analysis
    blob = TextBlob(lyrics)
    polarity_score = blob.sentiment.polarity
    
    return polarity_score

# List of files containing song lyrics
files = [
    "frank_ocean_nikes_lyrics.json",
    "frank_ocean_nights_lyrics.json",
    "frank_ocean_pink_white_lyrics.json",
    "frank_ocean_pyramids_lyrics.json"
]

# Perform sentiment analysis on each file and print the polarity score
for file in files:
    polarity_score = analyze_sentiment_from_file(file)
    print(f"Polarity score for {file}: {polarity_score}")


Polarity score for frank_ocean_nikes_lyrics.json: -0.019534050179211473
Polarity score for frank_ocean_nights_lyrics.json: -0.017717717717717723
Polarity score for frank_ocean_pink_white_lyrics.json: -0.02746913580246914
Polarity score for frank_ocean_pyramids_lyrics.json: 0.1732323232323232


For the top three songs, I believe that the polarity scores do match.  I don't agree with the score given on the final song as I think it's a bit more negative.  